<a href="https://colab.research.google.com/github/CodeVoyager2006/recipe-ai/blob/main/GenAI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gen A.I Assignment
by: Minh Son Truong/shen.truong2017@gmail.com
##Setup Phase
import the important libraries from Huggingface

In [16]:
!pip install transformers datasets

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load the flan-t5 model and tokenize it
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

cpu


## Set up the method to use the A.I model

In [19]:
def recipe(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
  recipe_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(recipe_ids[0], skip_special_tokens=True)

In [20]:
# Define a sample text for summarization
sample_text =     """
chicken, pasta, tomato sauce
"""

# Summarize the sample text using the pre-trained model (without fine-tuning)
pre_finetuned_recipe_generation = recipe(sample_text)
print("recipe generation before fine-tuning:", pre_finetuned_recipe_generation)

recipe generation before fine-tuning: chicken, pasta, tomato sauce


## Load dataset
- First import the library needed to load the dataset from hugging face


- Then, load the database that will be used

In [21]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset, which contains articles and summaries
dataset = load_dataset("Shengtao/recipe", split="train")

## Tokenize
tokenize the necessary data from the database

In [22]:
def preprocess_function(examples):
  # Extract the articles from the dataset
  inputs = [ingredient for ingredient in examples['ingredients']]

  # Tokenize the articles (inputs) with padding and truncation to a max length of 512
  model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

  # Tokenize the summaries (labels) using the target tokenizer context
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['title'], max_length=128, padding="max_length", truncation=True, return_tensors="pt")

  # Attach the tokenized summaries as labels to the model inputs
  model_inputs["labels"] = labels["input_ids"]

  # Move the tokenized inputs and labels to the appropriate device (GPU/CPU)
  model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

  return model_inputs

In [ ]:
# Tokenize the small training dataset
tokenized_train_dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/32722 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Set up training arguments

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Directory to save the model checkpoints
    evaluation_strategy="epoch",         # Evaluate the model at the end of every epoch
    learning_rate=2e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    weight_decay=0.01,                   # Regularization to prevent overfitting
    save_total_limit=3,                  # Only keep the last 3 checkpoints
    num_train_epochs=3,                  # Number of training epochs
    predict_with_generate=True,          # Enable text generation during evaluation
    logging_dir="./logs"                 # Directory for storing training logs
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Set up the trainer object
Using the Seq2SeqTrainer, we can train our model

In [ ]:
from transformers import Seq2SeqTrainer

# Create the trainer object
trainer = Seq2SeqTrainer(
    model=model,                         # The model to be trained
    args=training_args,                  # The training arguments defined earlier
    train_dataset=tokenized_train_dataset,  # The tokenized training dataset
    tokenizer=tokenizer                  # The tokenizer to handle input and output
)

In [ ]:
# Let's train
trainer.train()

KeyboardInterrupt: 

## Evaluation
Evaluate the metrics belong to our model post train

In [ ]:
# Evaluate the model on the evaluation dataset
metrics = trainer.evaluate()

# Print the evaluation metrics
print(metrics)

{'eval_loss': 42.93536376953125, 'eval_runtime': 0.1018, 'eval_samples_per_second': 19.643, 'eval_steps_per_second': 9.821, 'epoch': 3.0}


## Final test
- Set up the recipe method again to be called
- Run test

In [ ]:
def recipe(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
  recipe_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
  return tokenizer.decode(recipe_ids[0], skip_special_tokens=True)

In [ ]:
print(recipe("""
1 (8 ounce) box elbow macaroni ; ¼ cup butter ; ¼ cup all-purpose flour ; ½ teaspoon salt ; ground black pepper to taste ; 2 cups milk ; 2 cups shredded Cheddar cheese
"""))